In [23]:
import pandas as pd
import re
import os
from fuzzywuzzy import fuzz
from openpyxl import load_workbook
from openpyxl.styles import PatternFill


In [24]:
def normalize_string(s):
    """Normalize string by converting to lowercase, handling NaN or float, and keeping spaces."""
    if isinstance(s, str):
        s = re.sub(r'[^a-zA-Z0-9\s=]', '', s.lower())  # Keep letters, numbers, spaces, and equal signs
        s = re.sub(r'\s+', ' ', s)  # Collapse multiple spaces into one
        return s.strip()
    else:
        return ''

In [25]:
def similarity_score(str1, str2):
    """
    Calculate token-based fuzzy similarity between two strings.
    100% means an exact match, 0% means completely different.
    """
    return fuzz.token_set_ratio(str1, str2)


In [26]:
def compare_encodings(
    study_file,
    encoding_column='encodings',
    field_label_column='field_label',
    cde_file='./KnowledgeBase/Compiled_CORE_CDEs list_English_one sheet_as of 2025-01-28.xlsx',
    study_sheet='Sheet1'
):
    """
    Compare study data dictionary encodings and field labels with HEAL CDE encodings using fuzzy token-based similarity.
    
    Parameters:
    - study_file: Path to the study data dictionary file (.xlsx or .csv).
    - encoding_column: Name of the column containing encodings in the study file.
    - field_label_column: Name of the column containing field labels in the study file.
    - cde_file: Path to the HEAL CDE knowledge base file (.xlsx).
    - study_sheet: Name of the sheet in the study file to process (default is 'Sheet1').
    
    Returns:
    - DataFrame: Original study data with match results.
    """

    # Load study data
    if study_file.endswith('.xlsx'):
        study_df = pd.read_excel(study_file, sheet_name=study_sheet)
    else:
        study_df = pd.read_csv(study_file)

    # Initialize new columns for matches
    study_df['Best Match CDE Name'] = None
    study_df['Best Match Score'] = None
    study_df['Best Match CRF Name'] = None

    study_df['Potential Match 2 - CDE Name'] = None
    study_df['Potential Match 2 - Score'] = None
    study_df['Potential Match 2 - CRF Name'] = None

    study_df['Potential Match 3 - CDE Name'] = None
    study_df['Potential Match 3 - Score'] = None
    study_df['Potential Match 3 - CRF Name'] = None

    # Normalize study encodings and field labels
    study_df['Normalized Combined'] = study_df.apply(
        lambda row: normalize_string(row[encoding_column] + " | " + row[field_label_column])
        if pd.notna(row[encoding_column]) and pd.notna(row[field_label_column]) else '',
        axis=1
    )

    # Load HEAL CDE encodings
    cde_df = pd.read_excel(cde_file, sheet_name='ALL')
    cde_df = cde_df.dropna(subset=['PV Description', 'Additional Notes (Question Text)'])

    # Normalize HEAL CDE encodings
    cde_df['Normalized Combined'] = cde_df['Additional Notes (Question Text)'].fillna('') + " | " + cde_df['PV Description'].fillna('')
    cde_df['Normalized Combined'] = cde_df['Normalized Combined'].apply(normalize_string)

    # Compare study encodings to CDEs
    for idx, row in study_df.iterrows():
        if row['Normalized Combined'] == '':
            continue

        best_match = None
        best_score = 0
        best_crf_name = None
        potential_matches = []

        for _, cde_row in cde_df.iterrows():
            score_percentage = similarity_score(row['Normalized Combined'], cde_row['Normalized Combined'])
            if score_percentage > best_score:
                best_match = cde_row['Variable Name']
                best_score = score_percentage
                best_crf_name = cde_row['CRF Name']
            potential_matches.append((cde_row['Variable Name'], score_percentage, cde_row['CRF Name']))

        # Sort potential matches by score
        potential_matches.sort(key=lambda x: x[1], reverse=True)

        # Remove duplicates
        unique_matches = []
        seen = set()
        for name, score, crf in potential_matches:
            if name not in seen:
                unique_matches.append((name, score, crf))
                seen.add(name)

        if best_match:
                unique_matches = [m for m in unique_matches if m[0] !=best_match]

        # Save best match
        if best_match:
            study_df.at[idx, 'Best Match CDE Name'] = best_match
            study_df.at[idx, 'Best Match Score'] = best_score
            study_df.at[idx, 'Best Match CRF Name'] = best_crf_name

        # Save top 2 other matches
        for i, (match_name, match_score, crf_name) in enumerate(unique_matches[:2], start=2):
            study_df.at[idx, f'Potential Match {i} - CDE Name'] = match_name
            study_df.at[idx, f'Potential Match {i} - Score'] = match_score
            study_df.at[idx, f'Potential Match {i} - CRF Name'] = crf_name

    # Save results
    output_base = os.path.basename(study_file).rsplit('.', 1)[0]  # Just filename, no path
    output_file = f"out/{output_base}_vlmd_cdesearch.xlsx"  # Force save into 'out' folder
    study_df.to_excel(output_file, index=False)
    print(f"Comparison complete. Results saved to {output_file}.")

    return output_file

In [27]:
if __name__ == "__main__":
    study_file = './in/SAMPLE_sprint_2020-12-16.xlsx'
    study_sheet = 'SheetA'
    encoding_column = 'Choices, Calculations, OR Slider Labels'
    field_label_column = 'Field Label'

    # --- Run your main function and capture output file path ---
    output_file = compare_encodings(
        study_file,
        encoding_column=encoding_column,
        field_label_column=field_label_column,
        study_sheet=study_sheet
    )

    # --- Now apply color coding safely ---
    from openpyxl import load_workbook
    from openpyxl.styles import PatternFill

    wb = load_workbook(output_file)
    ws = wb.active

    # Find "Best Match Score" column
    best_match_score_col = None
    for idx, cell in enumerate(ws[1], start=1):
        if cell.value == "Best Match Score":
            best_match_score_col = idx
            break

    if best_match_score_col:
        for row in ws.iter_rows(min_row=2, min_col=best_match_score_col, max_col=best_match_score_col):
            for cell in row:
                if cell.value is not None:
                    if cell.value >= 80:
                        cell.fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
                    elif 51 <= cell.value <= 79:
                        cell.fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")
                    else:
                        cell.fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")

    wb.save(output_file)

    print(f"Color coding applied to {output_file}!")


Comparison complete. Results saved to out/SAMPLE_sprint_2020-12-16_vlmd_cdesearch.xlsx.
Color coding applied to out/SAMPLE_sprint_2020-12-16_vlmd_cdesearch.xlsx!


# HEAL CDE Fuzzy Matching Script

This script automates the matching of study data dictionaries to the HEAL Core Common Data Elements (CDEs), using smart fuzzy text comparison and a color-coded Excel output for easier review.

It is designed to help identify the **Best Match** and **Top 2 Potential Matches** for each study variable — even if the wording or encoding order is slightly different from the CDE standard.

---

## ✨ Key Features
- **Fuzzy Matching**: Uses token-based similarity (`Token Set Ratio`) to handle small typos and different word orders.
- **Normalized Comparisons**: Cleans and standardizes text for reliable matching.
- **Separate Output Folder**: All results are saved neatly into an `/out/` subfolder.
- **Color Coded Scores**:  
  - 🟩 **Green** for matches ≥ 80%  
  - 🟧 **Orange** for matches 51–79%  
  - 🟥 **Red** for matches ≤ 50%
- **No Duplicate Matches**: Ensures Best Match and Potential Matches are truly different.

---

## 🚀 How It Works

1. **Input**:
   - A study data dictionary (Excel `.xlsx` or CSV `.csv` file).
   - The master HEAL CDEs file (Excel file).

2. **Process**:
   - Normalize (clean) text by lowercasing, removing special characters, and preserving logical structures like equal signs.
   - Compare the study's "Encoding + Field Label" to the CDE's "PV Description + Question Text".
   - Select the best match and top two alternatives based on fuzzy matching scores.
   - Color-code the best match scores for easy visualization.

3. **Output**:
   - A new Excel file saved into `/out/`, with best matches listed and scores color-coded.

---

## 🛠️ Requirements

Install the following Python packages:

```bash
pip install pandas openpyxl fuzzywuzzy
```

---

## 📂 Folder Structure

```
/in/         # Input study files
/out/        # Output matched files (automatically created if not present)
/KnowledgeBase/ # Contains the HEAL Core CDE file
script.py    # Your main script
```

---

## 📋 Usage Example

```bash
python script.py
```

The output file will appear in the `/out/` folder and will be named something like:

```
SAMPLE_sprint_2020-12-16_vlmd_cdesearch.xlsx
```

---

## 🧠 Notes

- The script **requires** the correct columns to be named in the study file (e.g., `Choices, Calculations, OR Slider Labels` and `Field Label`).
- Only the **Best Match Score** column is color coded for quick review.
- Ensure your HEAL CDE master file contains the necessary columns: `PV Description` and `Additional Notes (Question Text)`.
